In [17]:
import torch
import torch.optim as opt
from sklearn.datasets import fetch_california_housing
import numpy as np
from MDN import MDN, SDN, distribution_log_prob_loss
from torch.utils.data import Dataset, DataLoader

In [4]:
num_steps = 5000
batch_size = 256
learning_rate = 1e-4
n_hidden = 4

device = "cuda" if torch.cuda.is_available() else "cpu"
data = fetch_california_housing()
X = torch.FloatTensor(data["data"]).to(device)
y = torch.FloatTensor(data["target"]).to(device)

In [24]:
model = SDN(X.shape[-1], n_hidden).to(device)
# model.load_state_dict(torch.load("run/sdn_state_dict.pt"))
optim = opt.Adam(model.parameters(), lr=learning_rate)

In [56]:
for i in range(num_steps):
    batch_idx = np.random.choice(X.shape[0], batch_size)
    dist = model(X[batch_idx])
    loss = distribution_log_prob_loss(dist, y[batch_idx])
    optim.zero_grad()
    loss.backward()
    optim.step()
    if i % 1000 == 0:
        print(loss.item())
torch.save(model.state_dict(), "run/sdn_state_dict.pt")

0.9008913040161133
0.8253390789031982
1.0706696510314941
0.8600386381149292
0.8180643320083618


In [12]:
model = MDN(X.shape[-1], n_hidden, 3).to(device)
# model.load_state_dict(torch.load("run/mdn_state_dict.pt"))
optim = opt.Adam(model.parameters(), lr=learning_rate)

In [13]:
for i in range(num_steps):
    batch_idx = np.random.choice(X.shape[0], batch_size)
    pi, dist = model(X[batch_idx])
    loss = distribution_log_prob_loss(dist, y[batch_idx, None], pi)
    optim.zero_grad()
    loss.backward()
    optim.step()
    if i % 1000 == 0:
        print(loss.item())
torch.save(model.state_dict(), "run/mdn_state_dict.pt")

3.2419862747192383
2.5416717529296875
2.093012809753418
1.8298964500427246
1.6176228523254395


In [57]:
class TensorDataset(Dataset):
    def __init__(self, data, target, return_index=False):
        if data.shape[0] != target.shape[0]:
            raise ValueError("Data and Target must have the same length")
        self.data = data
        self.target = target
        self.return_index = return_index

    def __len__(self):
        return self.data.shape[0]

    def __getitem__(self, idx):
        if idx < 0:
            idx = len(self) + idx
        if self.return_index:
            return idx, self.data[idx], self.target[idx]
        return self.data[idx], self.target[idx]

dataloader = DataLoader(TensorDataset(X, y, return_index=True), batch_size=batch_size, shuffle=False)
model.eval()
threshold = 2
possible_houses = []
with torch.no_grad():
    for i_batch, (idx_batched, X_batched, y_batched) in enumerate(dataloader):
        dist = model(X_batched)
        possible_houses.append(idx_batched[dist.mean + 1.28*dist.stddev <= threshold])
possible_houses = torch.hstack(possible_houses)

In [58]:
print(possible_houses.shape)


torch.Size([5478])
